# Data loading

In [ ]:
from zipfile import ZipFile
train = './train.zip'

with ZipFile(train, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
from zipfile import ZipFile
test = './test.zip'

with ZipFile(test, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision.io import read_image
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
import torch.nn.functional as F
import torchvision
import math

# Data preprossing

In [6]:
# Data preprocessing
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [7]:
train_dir = './dataset_slova/train'
test_dir = './dataset_slova/test'
print("done")

done


In [8]:
# Create datasets
train_dataset = ImageFolder(train_dir, transform=transform)
test_dataset = ImageFolder(test_dir, transform=transform)

# Training

In [9]:
# Hyperparameters
batch_size = 16
num_epochs = 10
learning_rate = 0.001

image_width = 475
image_height = 100
# Convolutional layer parameters
kernel_size = 3
stride = 1
padding = 1
channels = 32

# Calculate output size after convolutional layer
conv_output_width = math.floor((image_width - kernel_size + 2 * padding) / stride) + 1
conv_output_height = math.floor((image_height - kernel_size + 2 * padding) / stride) + 1

# Calculate output size after max pooling layer
pool_output_width = math.floor(conv_output_width / 2)
pool_output_height = math.floor(conv_output_height / 2)
print(pool_output_width, pool_output_height)

237 50


In [10]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(pool_output_width * pool_output_height * channels, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, pool_output_width * pool_output_height * channels)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
model = CNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
def train(model, train_loader, optimizer, criterion):
    train_loss = 0

    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        #print(outputs.shape)
        outputs = torch.squeeze(outputs)
        #print(outputs.shape)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()*inputs.size(0)

    return train_loss

def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            #print(outputs.shape)
            outputs = torch.squeeze(outputs)
            #print(outputs.shape)
            predicted = (outputs > 0).int()  # Convert logits to binary predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'total: {total}, correct: {correct}')
    return accuracy


In [14]:
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    accuracy = evaluate(model, test_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Test Accuracy: {accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'binary_classifier_cnn.pth')

total: 240, correct: 163
Epoch 1/10, Test Accuracy: 0.6792
total: 240, correct: 192
Epoch 2/10, Test Accuracy: 0.8000
total: 240, correct: 202
Epoch 3/10, Test Accuracy: 0.8417
total: 240, correct: 191
Epoch 4/10, Test Accuracy: 0.7958
total: 240, correct: 211
Epoch 5/10, Test Accuracy: 0.8792
total: 240, correct: 199
Epoch 6/10, Test Accuracy: 0.8292
total: 240, correct: 202
Epoch 7/10, Test Accuracy: 0.8417
total: 240, correct: 209
Epoch 8/10, Test Accuracy: 0.8708
total: 240, correct: 209
Epoch 9/10, Test Accuracy: 0.8708
total: 240, correct: 212
Epoch 10/10, Test Accuracy: 0.8833
